# Imports

In [2]:
# Global imports
import os
import numpy as np
import pandas as pd
import imageio.v2 as imageio
import tensorflow as tf
import tensorflow_addons as tfa
import seaborn as sns
import matplotlib.pyplot as plt
import librosa
import glob
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, KFold
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import BatchNormalization, Conv2D, LeakyReLU, MaxPooling2D, Flatten, Dense
import torchaudio

In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# Loading the data

## Global data

We first load the CSV file with the data of all participants, then we will use it to make a wider dataset with all the recordings

In [4]:
df = pd.read_csv('D:\Github\Improving-deep-neural-networks-to-identify-mental-disorders-using-Neural-Architecture-Search\D3T3C\D3TEC Dataset\Dataset.csv')
df.head()

,Marca temporal,Participant_ID,PHQ-9 Score,Age,Gender,Lugar de Residencia,Lugar de Procedencia,Social Class,Institution,Medicine,Physical Condition,Mental Health Condition,Depression Diagnosis (level)
0,2023/10/11 11:13:07 a. m. GMT-6,1,8,36,Female,"Santa Catarina, Nuevo León",NaN,Working Class,CAABI,Forxiga. Atrovastatina.,Diabetes. Fatty Liver.,NaN,NaN
1,2023/10/11 1:13:47 p. m. GMT-6,2,1,57,Female,"Santa Catarina, Nuevo León",NaN,Working Class,CAABI,Metformina,NaN,NaN,NaN
2,2023/10/11 1:57:08 p. m. GMT-6,3,3,62,Female,"San Pedro, Nuevo León",NaN,Upper Class,CAABI,Sertrialina,Estenosis Espinal,Ansiedad,NaN
3,2023/10/11 3:02:28 p. m. GMT-6,4,13,48,Female,"Santa Catarina, Nuevo León",NaN,Working Class,CAABI,NaN,NaN,NaN,NaN
4,2023/10/11 4:11:32 p. m. GMT-6,5,7,40,Female,"Santa Catarina, Nuevo León",NaN,Middle Class,CAABI,Homeopatía.,NaN,Moderate Anxiety.,NaN


### Checking types

In [5]:
df.dtypes

Marca temporal                  object
Participant_ID                   int64
PHQ-9 Score                      int64
Age                              int64
Gender                          object
Lugar de Residencia             object
Lugar de Procedencia            object
Social Class                    object
Institution                     object
Medicine                        object
Physical Condition              object
Mental Health Condition         object
Depression Diagnosis (level)    object
dtype: object

### Adding binary classification

In [6]:
# Crear la nueva columna "PHQ-Binary" basada en "PHQ-9 Score"
df['PHQ-Binary'] = df['PHQ-9 Score'].apply(lambda x: 1 if x >= 10 else 0)
df

,Marca temporal,Participant_ID,PHQ-9 Score,Age,Gender,Lugar de Residencia,Lugar de Procedencia,Social Class,Institution,Medicine,Physical Condition,Mental Health Condition,Depression Diagnosis (level),PHQ-Binary
0,2023/10/11 11:13:07 a. m. GMT-6,1,8,36,Female,"Santa Catarina, Nuevo León",NaN,Working Class,CAABI,Forxiga. Atrovastatina.,Diabetes. Fatty Liver.,NaN,NaN,0
1,2023/10/11 1:13:47 p. m. GMT-6,2,1,57,Female,"Santa Catarina, Nuevo León",NaN,Working Class,CAABI,Metformina,NaN,NaN,NaN,0
2,2023/10/11 1:57:08 p. m. GMT-6,3,3,62,Female,"San Pedro, Nuevo León",NaN,Upper Class,CAABI,Sertrialina,Estenosis Espinal,Ansiedad,NaN,0
3,2023/10/11 3:02:28 p. m. GMT-6,4,13,48,Female,"Santa Catarina, Nuevo León",NaN,Working Class,CAABI,NaN,NaN,NaN,NaN,1
4,2023/10/11 4:11:32 p. m. GMT-6,5,7,40,Female,"Santa Catarina, Nuevo León",NaN,Middle Class,CAABI,Homeopatía.,NaN,Moderate Anxiety.,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,2024/04/09 5:28:51 p. m. GMT-6,149,12,19,Male,"Monterrey, Nuevo León.","Bogotá, Colombia.",Upper Class,TEC,NaN,NaN,NaN,NaN,1
58,2024/04/10 4:17:26 p. m. GMT-6,150,12,20,Female,"Monterrey, Nuevo León.",San Luis Potosí,Middle Class,TEC,NaN,NaN,NaN,NaN,1
59,2024/04/10 4:18:06 p. m. GMT-6,151,4,20,Male,"Monterrey, Nuevo León.","Monterrey, Nuevo León.",Upper Class,TEC,NaN,NaN,NaN,NaN,0
60,2024/04/10 4:18:50 p. m. GMT-6,152,5,22,Male,"Monterrey, Nuevo León.",Oaxaca de Juárez,Upper Class,TEC,NaN,NaN,NaN,NaN,0


### Treat data frame as dict

In [7]:
df_dict = df.set_index('Participant_ID').to_dict('index')
df_dict

{1: {'Marca temporal': '2023/10/11 11:13:07 a.\xa0m. GMT-6',
  'PHQ-9 Score': 8,
  'Age': 36,
  'Gender': 'Female',
  'Lugar de Residencia': 'Santa Catarina, Nuevo León',
  'Lugar de Procedencia': nan,
  'Social Class': 'Working Class',
  'Institution': 'CAABI',
  'Medicine': 'Forxiga. Atrovastatina.',
  'Physical Condition': 'Diabetes. Fatty Liver.',
  'Mental Health Condition': nan,
  'Depression Diagnosis (level)': nan,
  'PHQ-Binary': 0},
 2: {'Marca temporal': '2023/10/11 1:13:47 p.\xa0m. GMT-6',
  'PHQ-9 Score': 1,
  'Age': 57,
  'Gender': 'Female',
  'Lugar de Residencia': 'Santa Catarina, Nuevo León',
  'Lugar de Procedencia': nan,
  'Social Class': 'Working Class',
  'Institution': 'CAABI',
  'Medicine': 'Metformina',
  'Physical Condition': nan,
  'Mental Health Condition': nan,
  'Depression Diagnosis (level)': nan,
  'PHQ-Binary': 0},
 3: {'Marca temporal': '2023/10/11 1:57:08 p.\xa0m. GMT-6',
  'PHQ-9 Score': 3,
  'Age': 62,
  'Gender': 'Female',
  'Lugar de Residencia': '

### Helper function definition

#### Spectrogram generator

In [10]:
import torchaudio
import torchaudio.transforms as T

def generate_mel_spectrogram(file_path, n_mels=128, n_fft=2048, hop_length=512):
    """
    Generate a mel spectrogram from an audio file.

    Args:
    - file_path (str): Path to the audio file.
    - n_mels (int): Number of mel filterbanks. Default is 128.
    - n_fft (int): Number of FFT components. Default is 2048.
    - hop_length (int): Length between successive frames. Default is 512.

    Returns:
    - mel_spectrogram (Tensor): The mel spectrogram.
    """
    # Load the audio file
    waveform, sample_rate = torchaudio.load(file_path)

    # Create a Mel Spectrogram Transform
    mel_spectrogram_transform = T.MelSpectrogram(
        sample_rate=sample_rate,
        n_fft=n_fft,
        hop_length=hop_length,
        n_mels=n_mels
    )

    # Apply the transform to get the mel spectrogram
    mel_spectrogram = mel_spectrogram_transform(waveform)

    return mel_spectrogram

In [9]:
# Map of valence according to the protocol questions
valence_map = {
    'positive': [0, 1, 2, 3, 4, 15, 18, 21],
    'neutral': [5, 6, 7, 8, 9, 16, 19, 22],
    'negative': [10, 11, 12, 13, 14, 17, 20, 23]
}

def load_audios(data_dict, base_path, audio_type):
    """
    Load audios for each participant according to the protocol and add them to the dictionary structure.
    
    Args:
    - data_dict (dict): Dictionary with the participants' information.
    - base_path (str): Base path where participants' audios are located.
    - audio_type (str): Type of audio ('sm' or 'mobile').
    
    Returns:
    - data_dict (dict): Updated dictionary with the loaded audios.
    """
    
    for participant_id, info in data_dict.items():
        # Create the structure for the audios
        info['audios'] = {
            'sm': {'positive': [], 'negative': [], 'neutral': []}
        }
        if audio_type not in info['audios']:
            info['audios'][audio_type] = {}
        
        # Path to the participant's audio folder
        #participant_audio_path = os.path.join(base_path, f'{participant_id:03d}')
        
        # Check if the folder exists
        # if not os.path.exists(participant_audio_path):
        #    print(f'Folder not found: {participant_audio_path}')
        #    continue
        
        # Load audios according to the protocol
        audio_files = os.listdir(base_path)
        for file in audio_files:
            if file == '.DS_Store':
                continue
            file_path = os.path.join(base_path, file)
            # Extract the question number from the file name
            if '_' in file:
                parts = file.split('_')
                if len(parts) == 2:
                    try:
                        question_number = int(parts[1].split('.')[0])
                    except:
                        question_number = int(parts[0].split('.')[0])
                elif len(parts) == 3:
                    question_number = int(parts[2].split('.')[0])
                else:
                    question_number = 0
            else:
                question_number = 0
            
            # Load the audio using torchaudio
            waveform, sample_rate = torchaudio.load(file_path)
            info['audios'][audio_type][question_number] = {
                'file_path': file_path,
                'waveform': waveform,
                'sample_rate': sample_rate
            }
            
            # Determine the valence according to the question number
            # Uncomment for valence classification
            """
            if question_number in valence_map['positive']:
                if 'positive' not in info['audios'][audio_type]:
                    info['audios'][audio_type]['positive'] = []
                info['audios'][audio_type]['positive'].append(file_path)
            elif question_number in valence_map['negative']:
                if 'negative' not in info['audios'][audio_type]:
                    info['audios'][audio_type]['negative'] = []
                info['audios'][audio_type]['negative'].append(file_path)
            elif question_number in valence_map['neutral']:
                if 'neutral' not in info['audios'][audio_type]:
                    info['audios'][audio_type]['neutral'] = []
                info['audios'][audio_type]['neutral'].append(file_path)
            """

    return data_dict


base_path_sm = 'D:/Github/Improving-deep-neural-networks-to-identify-mental-disorders-using-Neural-Architecture-Search/D3T3C/D3TEC Dataset/SM-27'
base_path_mobile = 'D:/Github/Improving-deep-neural-networks-to-identify-mental-disorders-using-Neural-Architecture-Search/D3T3C/D3TEC Dataset/iPhoneSE2020'

data_dict_updated = load_audios(df_dict, base_path_sm, 'sm')
print('---------------------')
data_dict_updated = load_audios(data_dict_updated, base_path_mobile, 'mobile')

# Verify the content of the updated dictionary
data_dict_updated


---------------------


{1: {'Marca temporal': '2023/10/11 11:13:07 a.\xa0m. GMT-6',
  'PHQ-9 Score': 8,
  'Age': 36,
  'Gender': 'Female',
  'Lugar de Residencia': 'Santa Catarina, Nuevo León',
  'Lugar de Procedencia': nan,
  'Social Class': 'Working Class',
  'Institution': 'CAABI',
  'Medicine': 'Forxiga. Atrovastatina.',
  'Physical Condition': 'Diabetes. Fatty Liver.',
  'Mental Health Condition': nan,
  'Depression Diagnosis (level)': nan,
  'PHQ-Binary': 0,
  'audios': {'sm': {'positive': [], 'negative': [], 'neutral': []},
   'mobile': {1: {'file_path': 'D:/Github/Improving-deep-neural-networks-to-identify-mental-disorders-using-Neural-Architecture-Search/D3T3C/D3TEC Dataset/iPhoneSE2020\\153_cel_1.wav',
     'waveform': tensor([[ 0.0000e+00, -2.6226e-06, -5.7220e-06,  ...,  1.0014e-05,
               6.3181e-06,  3.0994e-06]]),
     'sample_rate': 44100},
    10: {'file_path': 'D:/Github/Improving-deep-neural-networks-to-identify-mental-disorders-using-Neural-Architecture-Search/D3T3C/D3TEC Dataset/

## SM-27

In [9]:
df_test = pd.DataFrame(data_dict_updated)

In [ ]:
df_sm =""

## iPhone SE 2020

In [ ]:
df_iphone=  ""

# Data processing

## Spectrograms